# Facility location problem

Determine location of new factories based on 
- transportation distance, costs 
- facility opening/maintenance cost
- demand
- capacitated : facilities have limited capcacity
- choose the best of a number of fixed locations    
    

See https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/facility_location/facility_location_gcl.ipynb

In [7]:
from math import sqrt
from itertools import product

In [8]:
# Parameters
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

In [9]:
# This function determines the Euclidean distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [10]:
# Compute key parameters of MIP model formulation

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

In [11]:
# Compute shipping costs

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

In [12]:
shipping_cost

{(0, 0): 1.5,
 (0, 1): 0.5,
 (0, 2): 0.5,
 (0, 3): 1.8027756377319946,
 (0, 4): 1.118033988749895,
 (0, 5): 1.118033988749895,
 (0, 6): 2.5,
 (0, 7): 2.0615528128088303,
 (0, 8): 2.0615528128088303,
 (1, 0): 2.773084924772409,
 (1, 1): 2.5079872407968904,
 (1, 2): 2.6248809496813377,
 (1, 3): 1.9209372712298547,
 (1, 4): 1.5132745950421556,
 (1, 5): 1.7,
 (1, 6): 1.3,
 (1, 7): 0.5385164807134504,
 (1, 8): 0.9433981132056605}

In [13]:
# Using SCIP for MIP problem (see https://developers.google.com/optimization/mip/mip_example)

# solver = pywraplp.Solver.CreateSolver('CBC_MIXED_INTEGER_PROGRAMMING')
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [14]:
select = {}
for j in range(num_facilities):
    select[j] = solver.BoolVar("Select")

In [15]:
assign = {}
for i,j in shipping_cost.keys(): 
    assign[(i,j)] = solver.NumVar(0,1,"Assign")

In [16]:
# shipping constraints
for i,j in cartesian_prod:
    solver.Add(assign[(i,j)] <= select[j])

In [17]:
# demand constraints
for i in range(num_customers):
    solver.Add(sum(assign[(i,j)] for j in range(num_facilities)) == 1)        

In [18]:
objective = solver.Objective()

In [19]:
for j in range(num_facilities):
    objective.SetCoefficient(select[j], setup_cost[j])

In [20]:
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],shipping_cost[(i,j)])

In [21]:
objective.SetMinimization()

In [22]:
status = solver.Solve()

In [23]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 4.723712908961849

Problem solved in 36.000000 milliseconds
Problem solved in 19 iterations
Problem solved in 1 branch-and-bound nodes


In [24]:
# display optimal values of decision variables

for facility in select.keys():
    if (abs(select[facility].solution_value()) > 1e-6):
        print(f"\n Build a warehouse at location {facility + 1}.")


 Build a warehouse at location 4.


In [25]:
# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].solution_value()) > 1e-6):
        print(f"\n Supermarket {customer + 1} receives {round(100*assign[customer, facility].solution_value(), 2)} % of its demand  from Warehouse {facility + 1} .")


 Supermarket 1 receives 100.0 % of its demand  from Warehouse 4 .

 Supermarket 2 receives 100.0 % of its demand  from Warehouse 4 .


# facility location problem 2
see https://scipbook.readthedocs.io/en/latest/flp.html

In [26]:
setup_cost = 1000
f_capacity = 500

demands = [80, 270, 250, 160, 180]
costs = {}
costs[0] = [4,5,6,8,10]
costs[1] = [6,4,3,5,8]
costs[2] = [9,7,4,3,4]

c = {}

for i in range(len(demands)):
    for j in costs.keys():
        c[(i,j)]=costs[j][i]

In [27]:
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [28]:
num_facilities = len(costs.keys())
num_customers = len(demands)

In [29]:
select = {}
for j in range(num_facilities):
    select[j] = solver.BoolVar("Select")

In [30]:
# boundaries
assign = {}
for i,j in c.keys(): 
    assign[(i,j)] = solver.NumVar(0,solver.infinity(),"Assign")

In [31]:
# shipping constraints
for i,j in c.keys():
    solver.Add(assign[(i,j)] <= select[j]*demands[i])

In [32]:
# demand constraints
for i in range(num_customers):
    solver.Add(sum(assign[(i,j)] for j in range(num_facilities)) == demands[i] )

In [33]:
# capacity constraints
for j in range(num_facilities):
    solver.Add(sum(assign[(i,j)] for i in range(num_customers)) <= select[j]*f_capacity)

In [34]:
objective = solver.Objective()

In [35]:
for j in range(num_facilities):
    objective.SetCoefficient(select[j], setup_cost)

In [36]:
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],c[(i,j)])

In [37]:
objective.SetMinimization()

In [38]:
status = solver.Solve()

In [39]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 5610.000000000001

Problem solved in 35.000000 milliseconds
Problem solved in 18 iterations
Problem solved in 1 branch-and-bound nodes


In [40]:
# display optimal values of decision variables

for facility in select.keys():
    if (abs(select[facility].solution_value()) > 1e-6):
        print(f"\n Build a warehouse at location {facility + 1}.")


 Build a warehouse at location 2.

 Build a warehouse at location 3.


In [41]:
# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].solution_value()) > 1e-6):
        print(f"\n Supermarket {customer + 1} receives {round(assign[customer, facility].solution_value(), 2)} of its demand {demands[customer]} from Warehouse {facility + 1} .")


 Supermarket 1 receives 80.0 of its demand 80 from Warehouse 2 .

 Supermarket 2 receives 270.0 of its demand 270 from Warehouse 2 .

 Supermarket 3 receives 150.0 of its demand 250 from Warehouse 2 .

 Supermarket 3 receives 100.0 of its demand 250 from Warehouse 3 .

 Supermarket 4 receives 160.0 of its demand 160 from Warehouse 3 .

 Supermarket 5 receives 180.0 of its demand 180 from Warehouse 3 .
